In [20]:
import torch
import sys
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse
import os
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

parser = argparse.ArgumentParser(description='This is a demonstration program')

# Here we add an argument to the parser, specifying the expected type, a help message, etc.
#parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch_size')

#args = parser.parse_args()

# Now we can use the argument value in our program.
#print(f'batch size: {args.batch_size}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 32
batch_size = 128
max_iters = 2000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

epoch = 5

modelfile = "model-01.pkl"
log_file = open('logs.txt', 'a') 

print(device)

cuda


In [11]:
chars = ""
with open("openwebtext/vocab.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))
        
vocab_size = len(chars)

In [12]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

#encode('hei')
#decode([74, 71, 75])


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
def get_random_chunk(split):
    filename = "openwebtext/train_split.txt" if split == 'train' else "openwebtext/val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [15]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)

#ny model
loadModel = False
if loadModel:
    print('loading model parameters...')
    with open(modelfile, 'rb') as f:
        model = pickle.load(f)
        print('loaded successfully!')
else:
    print('running new model')    
m = model.to(device)

running new model


In [16]:
def colored(text, color):
    colors = {
        "red": "\033[91m",
        "green": "\033[92m",
        "yellow": "\033[93m",
        "blue": "\033[94m",
        "magenta": "\033[95m",
        "cyan": "\033[96m"
    }
    return colors.get(color, "") + text + "\033[0m"

In [19]:
def log_and_print(message):
    print(message)
    log_file.write(message + '\n')

In [18]:
#PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for repeat in range(epoch):
    start_time = time.time()

    log_and_print("Starting script iteration " + str(repeat + 1) + " at: " + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time)))
    log_and_print(f"Block Size: {block_size}, Batch Size: {batch_size}")
    log_and_print(f"Number of Heads: {n_head}, Number of Layers: {n_layer}")
    log_and_print(f"Max Iterations: {max_iters}, Eval Iters: {eval_iters}")
    log_and_print(f"Learning Rate: {learning_rate}")
    log_and_print(f"Embedding Size: {n_embd}, Dropout: {dropout}")
    # If you want to reset optimizer and loss variables for each iteration, initialize them here
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    prev_train_loss = None
    prev_val_loss = None
    for iter in range(max_iters):
        # Record the start time of each iteration
        iter_start_time = time.time()
    
        if iter % eval_iters == 0:
            losses = estimate_loss()
    
            # Calculate the difference in losses
            train_loss_diff = losses['train'] - prev_train_loss if prev_train_loss is not None else 0
            val_loss_diff = losses['val'] - prev_val_loss if prev_val_loss is not None else 0
    
            # Update the previous losses
            prev_train_loss = losses['train']
            prev_val_loss = losses['val']
    
            log_and_print(f"step: {iter}, train loss: {losses['train']:.3f}({train_loss_diff:+.3f}), val loss: {losses['val']:.3f}({val_loss_diff:+.3f})")
    
        # sample a batch of data
        xb, yb = get_batch('train')
    
        # evaluate the loss
        logits, loss = model.forward(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    
        # Record the end time of each iteration and calculate the time taken
        iter_end_time = time.time()
        # print(f"Iteration {iter} completed in {iter_end_time - iter_start_time:.2f} seconds")
    
    log_and_print(loss.item())
    
    # script for 10000 iterations here
    
    with open('model-01.pkl', 'wb') as f:
        pickle.dump(model, f)
    print('model saved')
    
    # End time of the script
    end_time = time.time()
    
    total_runtime_minutes = (end_time - start_time) / 60
    
    #log_and_print("Ending script at:", time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(end_time)))
    log_and_print(f"Total script runtime: {total_runtime_minutes:.2f} minutes")

log_file.close()

Starting script iteration 1 at: 2023-11-18 21:13:02
Block Size: 32, Batch Size: 128
Number of Heads: 4, Number of Layers: 4
Max Iterations: 2000, Eval Iters: 100
Learning Rate: 0.0003
Embedding Size: 384, Dropout: 0.2
step: 0, train loss: 10.517(+0.000), val loss: 10.505(+0.000)
step: 100, train loss: 2.370(-8.147), val loss: 2.435(-8.070)
step: 200, train loss: 2.234(-0.136), val loss: 2.259(-0.176)
step: 300, train loss: 2.137(-0.097), val loss: 2.242(-0.017)
step: 400, train loss: 1.996(-0.141), val loss: 2.045(-0.198)
step: 500, train loss: 1.943(-0.052), val loss: 1.911(-0.133)
step: 600, train loss: 1.882(-0.061), val loss: 1.897(-0.015)
step: 700, train loss: 1.888(+0.006), val loss: 1.932(+0.036)
step: 800, train loss: 1.852(-0.036), val loss: 1.890(-0.043)
step: 900, train loss: 1.894(+0.043), val loss: 1.799(-0.091)
step: 1000, train loss: 1.780(-0.114), val loss: 1.792(-0.007)
step: 1100, train loss: 1.782(+0.002), val loss: 1.763(-0.028)
step: 1200, train loss: 1.746(-0.036

TypeError: can only concatenate str (not "float") to str